# Basic Image Retrieval

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
ROOT = 'data'
CLASS_NAME = sorted(list(os.listdir(f'{ROOT}/train')))

In [ ]:
def read_image_from_path(path, size):
    im = Image.open(path).convert('RGB').resize(size)
    return np.array(im)

def folder_to_images(folder, size):
    list_dir = [folder + '/' + name for name in os.listdir(folder)]
    images_np = np.zeros(shape=(len(list_dir), *size, 3))
    images_path = []

    for i, path in enumerate(list_dir):
        images_np[i] = read_image_from_path(path, size)
        images_path.append(path)
    return images_np, images_path

In [ ]:
def absolute_diffirence(query, data):
    axis_batch_size = tuple(range(1, len(data.shape)))
    return np.sum(np.abs(query - data), axis=axis_batch_size)

In [ ]:
def get_l1_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    ls_path_score = []

    for folder in os.listdir(root_img_path):
        if folder in CLASS_NAME:
            path = root_img_path + folder
            images_np, images_path = folder_to_images(path, size)
            rates = absolute_diffirence(query, images_np)
            ls_path_score.extend(list(zip(images_path, rates)))

    return query, ls_path_score

## l1 Score

In [ ]:
def plot_results(query_path, ls_path_score, reverse=False):
  """
  Plot images from a list of paths.

  Args:
    query_path: Path to the query image.
    ls_path_score: A list of tuples containing (image_path, score).
    reverse: If True, sort the images in descending order of score.
  """
  # Sort the images by score
  ls_path_score_sorted = sorted(ls_path_score, key=lambda x: x[1], reverse=reverse)

  # Create a figure and axes
  fig, axes = plt.subplots(nrows=1, ncols=len(ls_path_score_sorted) + 1, figsize=(20, 5))

  # Plot the query image
  query_image = Image.open(query_path)
  axes[0].imshow(query_image)
  axes[0].set_title('Query Image')
  axes[0].axis('off')

  # Plot the other images
  for i, (path, score) in enumerate(ls_path_score_sorted):
    image = Image.open(path)
    axes[i + 1].imshow(image)
    axes[i + 1].set_title(f'Score: {score:.2f}')
    axes[i + 1].axis('off')

  plt.tight_layout()
  plt.show()


In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## l2 Score

In [ ]:
def mean_square_diffirence(query, data):
    axis_batch_size = tuple(range(1, len(data.shape)))
    return np.sum(np.square(query - data), axis=axis_batch_size)

In [ ]:
def get_l2_score(root_img_path, query_path, size):
    query = read_image_from_path(query)
    ls_path_score = []

    for folder in os.listdir(root_img_path):
        if folder in CLASS_NAME:
            path = root_img_path + folder
            images_np, images_path = folder_to_images(path, size)
            rates = mean_square_diffirence(query, images_np)
            ls_path_score.extend(list(zip(images_path, rates)))

    return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l2_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l2_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## Cosine Similarity

In [ ]:
def cosine_similarity(query, data):
    axis_batch_size = tuple(range(1, len(data.shape)))
    query_norm = np.sqrt(np.sum(query**2))
    data_norm = np.sqrt(np.sum(data**2, axis=axis_batch_size))
    return np.sum(query * data, axis=axis_batch_size) / (query_norm * data_norm + np.finfo(np.float32).eps)

In [ ]:
def get_cosine_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    ls_path_score = []

    for folder in os.listdir(root_img_path):
        if folder in CLASS_NAME:
            path = root_img_path + folder
            images_np, images_path = folder_to_images(path, size)
            rates = cosine_similarity(query, images_np)
            ls_path_score.extend(list(zip(images_path, rates)))

    return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_cosine_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## Correlation Coeffient

In [ ]:
def correlation_coffient(query, data):
    axis_batch_size = tuple(range(1, len(data.shape)))
    query_mean = query - np.mean(query)
    data_mean = data - np.mean(data, axis=axis_batch_size, keepdims=True)
    query_norm = np.sqrt(np.sum(query_mean**2))
    data_norm = np.sqrt(np.sum(data_mean**2, axis=axis_batch_size))

    return np.sum(data_mean * query_mean, axis=axis_batch_size) / (query_norm * data_norm + np.finfo(np.float32).eps)


In [ ]:
def get_correlation_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    ls_path_score = []

    for folder in os.listdir(root_img_path):
        if folder in CLASS_NAME:
            path = root_img_path + folder
            images_np, images_path = folder_to_images(path, size)
            rates = correlation_coffient(query, images_np)
            ls_path_score.extend(list(zip(images_path, rates)))

    return query, ls_path_score

# Advanced Image Retrieval

##

In [ ]:
!pip install -q chromadb
!pip install -q open-clip-torch

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from chromadb.utils.embedding_functions.open_clip_embedding_function import OpenCLIPEmbeddingFunction

## l1 Score

In [ ]:
embedding_function = OpenCLIPEmbeddingFunction()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
def get_single_image_embedding(image):
    embedding = embedding_function._encode_image(image)
    return np.array(embedding)

In [ ]:
def get_l1_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    ls_path_score = []

    for folder in os.listdir(root_img_path):
        if folder in CLASS_NAME:
            path = root_img_path + folder
            images_np, images_path = folder_to_images(path, size)

            embedding_list = []
            for idx_img in range(images_np.shape[0]):
                embedding = get_single_image_embedding(images_np[idx_img])
                embedding_list.append(embedding)

            rates = absolute_diffirence(query, embedding_list)
            ls_path_score.extend(list(zip(images_path, rates)))

    return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## l2 Score

In [ ]:
def get_l2_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    query_embedding = get_single_image_embedding(query)
    ls_path_core = []

    for folder in os.listdir(root_img_path):
        path = root_img_path + folder
        images_np, images_path = folder_to_images(path, size)

        embedding_list = []
        for idx_img in range(images_np.shape[0]):
            embedding = get_single_image_embedding(images_np[idx_img])
            embedding_list.append(embedding)

        rates = mean_square_diffirence(query_embedding, embedding_list)
        ls_path_core.extend(list(zip(images_path, rates)))

    return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## Correlation Coeffient

In [ ]:
def get_correlation_coeffient_score(root_img_path, query_path, size):
    query = read_image_from_path(query_path, size)
    query_embedding = get_single_image_embedding(query)
    ls_path_core = []

    for folder in os.listdir(root_img_path):
        path = root_img_path + folder
        images_np, images_path = folder_to_images(path, size)

        embedding_list = []
        for idx_img in range(images_np.shape[0]):
            embedding = get_single_image_embedding(images_np[idx_img])
            embedding_list.append(embedding)

        rates = correlation_coffient(query_embedding, embedding_list)
        ls_path_core.extend(list(zip(images_path, rates)))

    return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, False)

## Using vector database

In [ ]:
def get_files_path(path):
    files_path = []

    for label in CLASS_NAME:
        label_path = path + "/" + label
        filenames = os.listdir(label_path)
        for filename in filenames:
            files_path.append(label_path + "/" + filename)

    return files_path

In [ ]:
data_path = f"{ROOT}/train"
files_path = get_files_path(data_path)

In [ ]:
def add_embedding(collection, files_path):
    ids = []
    embeddings = []

    for id_filepath, filepath in tqdm(enumerate(files_path), total=len(files_path)):
        ids.append(f"id_{id_filepath}")
        image = Image.open(filepath)
        embedding = get_single_image_embedding(image)
        embeddings.append(embedding)

    collection.add(
        embeddings=embeddings,
        ids=ids
    )

In [ ]:
# Create a Chroma Client
import chromadb
chroma_client = chromadb.Client()

# Create a collection
l2_collection = chroma_client.create_collection(name="l2_collection", metadata={HNSW_SPACE: "l2"})
add_embedding(l2_collection, files_path)

In [ ]:
def search(image_path, collection, n_results=5):
    query_image = Image.open(image_path)
    query_embedding = get_single_image_embedding(query_image)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return results

In [ ]:
def plot_results(image_path, files_path, results, n_results=5):
  """
  Plot images from a list of paths.

  Args:
    image_path: Path to the query image.
    files_path: A list of paths to all images in the dataset.
    results: The results of a ChromaDB query.
    n_results: The number of results to display.
  """
  # Get the IDs of the retrieved images
  retrieved_image_ids = results['ids'][0]

  # Create a figure and axes
  fig, axes = plt.subplots(nrows=1, ncols=n_results + 1, figsize=(20, 5))

  # Plot the query image
  query_image = Image.open(image_path)
  axes[0].imshow(query_image)
  axes[0].set_title('Query Image')
  axes[0].axis('off')

  # Plot the retrieved images
  for i in range(n_results):
    retrieved_image_id = retrieved_image_ids[i]
    retrieved_image_index = int(retrieved_image_id.split('_')[1])
    retrieved_image_path = files_path[retrieved_image_index]
    image = Image.open(retrieved_image_path)
    axes[i + 1].imshow(image)
    axes[i + 1].set_title(f'Result {i+1}')
    axes[i + 1].axis('off')

  plt.tight_layout()
  plt.show()


In [ ]:
test_path = f"{ROOT}/train"
test_files_path = get_files_path(test_path)
test_path = test_files_path[1]
l2_results = search(image_path=test_path, collection=l2_collection, n_results=5)
plot_results(image_path=test_path, files_path=files_path, results=l2_results)

In [ ]:
cosine_collection = chroma_client.get_or_create_collection(name="Cosine_collection",
                                                           metadata={HNSW_SPACE: "cosine"})
add_embedding(collection=cosine_collection, files_path=files_path)

1. C
2. A
3. B
4. A
5. D
6. B
7. D
8. B
9. B